<a href="https://colab.research.google.com/github/darthfelipe21/python_practice/blob/main/Lambda_aws_counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import boto3
import datetime
import pytz
import pymysql

# Configuración de la conexión
def get_db_connection():
    return pymysql.connect(host='tu-host',
                           user='tu-usuario',
                           password='tu-contraseña',
                           database='tu-base-de-datos')

# Función Lambda para contar clientes nuevos por día
def lambda_handler(event, context):
    # Obtener la hora actual 
    current_time = datetime.datetime.now(pytz.timezone('America/Santiago'))

    # Activar a medianoche local
    if current_time.hour == 0 and current_time.minute == 0:
        # Conecta a la base de datos
        connection = get_db_connection()

        # Fecha de ayer 
        yesterday = current_time - datetime.timedelta(days=1)
        yesterday_date = yesterday.strftime('%Y-%m-%d')

        # Consulta SQL 
        query = f"SELECT COUNT(*), nombre, correo_electronico FROM clientes WHERE fecha_registro >= '{yesterday_date} 00:00:00' AND fecha_registro <= '{yesterday_date} 23:59:59'"

        # Ejecutar la consulta y obtener resultado
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
            new_customers_count = result[0][0]
            new_customers = [
                {
                    'nombre': customer[1],
                    'correo_electronico': customer[2]
                } for customer in result
            ]

        # Cerrar la conexión a la base de datos
        connection.close()

        # Resultado
        return {
            'statusCode': 200,
            'body': f'Se encontraron {new_customers_count} clientes nuevos el {yesterday_date}.'
            'new_customers': new_customers
        }
    else:
        # Si no es medianoche
        return {
            'statusCode': 200,
            'body': 'La función se ejecuta a medianoche.'
        }
